## Part 1: Preprocessing

In [24]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf



#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [4]:
# Determine the number of unique values in each column.
attrition_df.nunique()

,0
Age,43
Attrition,2
BusinessTravel,3
Department,3
DistanceFromHome,29
Education,5
EducationField,6
EnvironmentSatisfaction,4
HourlyRate,71
JobInvolvement,4


In [6]:
# Correct code: Use attrition_df instead of df
y_df = attrition_df[['Attrition', 'Department']]


In [12]:
# List all column names
attrition_df.columns



Index(['Age', 'Attrition', 'BusinessTravel', 'Department', 'DistanceFromHome',
       'Education', 'EducationField', 'EnvironmentSatisfaction', 'HourlyRate',
       'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction',
       'MaritalStatus', 'NumCompaniesWorked', 'OverTime', 'PercentSalaryHike',
       'PerformanceRating', 'RelationshipSatisfaction', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager'],
      dtype='object')

In [13]:
# Correct feature selection with valid column names
X_df = attrition_df[['Age', 'DistanceFromHome', 'Education', 'EnvironmentSatisfaction',
                     'JobSatisfaction', 'HourlyRate', 'NumCompaniesWorked',
                     'PercentSalaryHike', 'StockOptionLevel', 'YearsAtCompany']]


In [14]:
# Show the data types for X_df
X_df.dtypes

,0
Age,int64
DistanceFromHome,int64
Education,int64
EnvironmentSatisfaction,int64
JobSatisfaction,int64
HourlyRate,int64
NumCompaniesWorked,int64
PercentSalaryHike,int64
StockOptionLevel,int64
YearsAtCompany,int64


In [15]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split



In [16]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=42)



In [17]:
# Create a StandardScaler
scaler = StandardScaler()
# Fit the scaler to the training data
X_train_scaled = scaler.fit_transform(X_train)
# Scale the testing data
X_test_scaled = scaler.transform(X_test)



In [21]:
# Create a OneHotEncoder for the Department column
encoder_department = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
y_train_department = encoder_department.fit_transform(y_train[['Department']])

# Transform the testing data
y_test_department = encoder_department.transform(y_test[['Department']])



In [22]:
# Create a OneHotEncoder for the Attrition column
encoder_attrition = OneHotEncoder(sparse_output=False)
# Fit the encoder to the training data
y_train_attrition = encoder_attrition.fit_transform(y_train[['Attrition']])
# Transform the testing data
y_test_attrition = encoder_attrition.transform(y_test[['Attrition']])


## Create, Compile, and Train the Model

In [25]:
# Find the number of columns in the X training data
input_layer = tf.keras.layers.Input(shape=(X_train_scaled.shape[1],))

In [26]:
# Create at least two shared layers
shared_layer = tf.keras.layers.Dense(64, activation='relu')(input_layer)
shared_layer = tf.keras.layers.Dense(32, activation='relu')(shared_layer)


In [27]:
# Create a branch for Department
department_hidden = tf.keras.layers.Dense(16, activation='relu')(shared_layer)
department_output = tf.keras.layers.Dense(y_train_department.shape[1], activation='softmax')(department_hidden)


In [28]:
# Create a branch for Attrition
attrition_hidden = tf.keras.layers.Dense(16, activation='relu')(shared_layer)
attrition_output = tf.keras.layers.Dense(y_train_attrition.shape[1], activation='sigmoid')(attrition_hidden)



In [29]:
# Create the model
model = tf.keras.Model(inputs=input_layer, outputs=[department_output, attrition_output])

# Compile the model
model.compile(optimizer='adam',
              loss=['categorical_crossentropy', 'binary_crossentropy'],
              metrics=['accuracy'])

# Summarize the model
model.summary()



Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 10)             │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense (Dense)             │ (None, 64)             │            704 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_1 (Dense)           │ (None, 32)             │          2,080 │ dense[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_2 (Dense)           │ (None, 16)             │            528 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_4 (Dense)           │ (None, 16)             │            528 │ dense_1[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_3 (Dense)           │ (None, 3)              │             51 │ dense_2[0][0]          │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_5 (Dense)           │ (None, 2)              │             34 │ dense_4[0][0]          │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 3,925 (15.33 KB)

 Trainable params: 3,925 (15.33 KB)

 Non-trainable params: 0 (0.00 B)

In [31]:
# Compile the model with separate metrics for each output
model.compile(optimizer='adam',
              loss=['categorical_crossentropy', 'binary_crossentropy'],  # One loss for each output
              metrics=[['accuracy'], ['accuracy']])  # Separate metrics for each output branch


In [32]:
# Train the model
history = model.fit(X_train_scaled,
                    [y_train_department, y_train_attrition],  # Provide both targets
                    validation_data=(X_test_scaled, [y_test_department, y_test_attrition]),
                    epochs=50,
                    batch_size=32)




Epoch 1/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step - dense_3_accuracy: 0.2423 - dense_5_accuracy: 0.6480 - loss: 1.8521 - val_dense_3_accuracy: 0.6667 - val_dense_5_accuracy: 0.8673 - val_loss: 1.4409
Epoch 2/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - dense_3_accuracy: 0.6545 - dense_5_accuracy: 0.8232 - loss: 1.4009 - val_dense_3_accuracy: 0.6667 - val_dense_5_accuracy: 0.8673 - val_loss: 1.2174
Epoch 3/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - dense_3_accuracy: 0.6602 - dense_5_accuracy: 0.8321 - loss: 1.2654 - val_dense_3_accuracy: 0.6667 - val_dense_5_accuracy: 0.8673 - val_loss: 1.1724
Epoch 4/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - dense_3_accuracy: 0.6462 - dense_5_accuracy: 0.8361 - loss: 1.2308 - val_dense_3_accuracy: 0.6667 - val_dense_5_accuracy: 0.8673 - val_loss: 1.1585
Epoch 5/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - dense_3_accuracy: 0.6395 - dense_5_accuracy: 0.8346 - loss: 1.2049 - val_dense_3_accuracy: 0.6667 - val_dense_5_accuracy: 0.8673 - val_loss: 1.1559

In [35]:
# Evaluate the model with the testing data
results = model.evaluate(X_test_scaled, [y_test_department, y_test_attrition])

# Debug: Print the full results array to understand its structure
print("Results:", results)



10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - dense_3_accuracy: 0.5913 - dense_5_accuracy: 0.8283 - loss: 1.4180 
Results: [1.3705161809921265, 0.6020408272743225, 0.8367347121238708]


In [36]:
# Print the accuracy for both department and attrition
print("Department predictions accuracy:", results[-2])  # Accuracy for department
print("Attrition predictions accuracy:", results[-1])  # Accuracy for attrition


Department predictions accuracy: 0.6020408272743225
Attrition predictions accuracy: 0.8367347121238708


In [39]:
# Evaluate the model with the testing data
results = model.evaluate(X_test_scaled, [y_test_department, y_test_attrition])

# Debug: Print the full results array to confirm structure
print("Results:", results)

# Print the accuracy for both department and attrition
print("Department predictions accuracy:", results[1])  # Accuracy for department
print("Attrition predictions accuracy:", results[2])  # Accuracy for attrition



10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - dense_3_accuracy: 0.5913 - dense_5_accuracy: 0.8283 - loss: 1.4180
Results: [1.3705161809921265, 0.6020408272743225, 0.8367347121238708]
Department predictions accuracy: 0.6020408272743225
Attrition predictions accuracy: 0.8367347121238708


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. Accuracy might not be the best metric for this data, particularly if there is a class imbalance.

For example: If the number of employees leaving the company (attrition) is much smaller than those staying, the model might predict "no attrition" for most cases and still achieve a high accuracy. However, this would not reflect the model's true performance.

A better metric could be precision, recall, or F1-score, depending on whether false positives or false negatives are more critical to minimize. For multi-class predictions like department assignment, accuracy is more reasonable, but confusion matrices could provide deeper insights

2. Department Output:

Activation Function: softmax

Reason: softmax is used for multi-class classification tasks because it assigns probabilities to each class, making sure the probabilities sum to 1. This is appropriate for predicting which department an employee belongs to.

Attrition Output:

Activation Function: sigmoid

Reason: sigmoid is used for binary classification tasks because it outputs a probability between 0 and 1, indicating the likelihood of an employee leaving the company (attrition).

3. Data Augmentation:

Augment the dataset if it’s small, especially if some departments or classes in attrition are underrepresented, to balance the data.

